# EIA 860M Update Inspection

To run this notebook, you need to refresh the changelog data first, by updating parameters:
- In `.env`, change `PUDL_VERSION` to the latest found [here](https://github.com/catalyst-cooperative/pudl/releases)
- In `src/constants.py`, set `PUDL_LATEST_YEAR` to the latest year for which PUDL has complete data.

and running `make all`.

In [1]:
import pandas as pd

from dbcp.helpers import get_sql_engine

engine = get_sql_engine()

with engine.connect() as con:
    eia860m = pd.read_sql_table("pudl_eia860m_changelog", con, schema="data_warehouse")
    pudl_eia860m_status_codes = pd.read_sql_table("pudl_eia860m_status_codes", con, schema="data_warehouse")

In [2]:
PREVIOUS_QUARTER_DATE = "2025-01-01"

def grab_current_quarter(df):
    recent_quarter = df.loc[df.groupby(["generator_id", "plant_id_eia"])['report_date'].idxmax()]
    assert ~recent_quarter.duplicated(subset=["generator_id", "plant_id_eia"]).any()
    return recent_quarter

def grab_previous_quarter(df):
    previous_quarter = df[df.report_date < PREVIOUS_QUARTER_DATE]
    previous_quarter = previous_quarter.loc[previous_quarter.groupby(["generator_id", "plant_id_eia"])['report_date'].idxmax()]
    assert ~previous_quarter.duplicated(subset=["generator_id", "plant_id_eia"]).any()
    return previous_quarter


def pct_change_mw_by_status(df):
    recent_quarter = grab_current_quarter(df)
    recent_quarter_mw_by_status = recent_quarter.groupby("operational_status_code").capacity_mw.sum()

    previous_quarter = grab_previous_quarter(df)
    
    previous_quarter_mw_by_status = previous_quarter.groupby("operational_status_code").capacity_mw.sum()

    return ((recent_quarter_mw_by_status - previous_quarter_mw_by_status) / previous_quarter_mw_by_status) * 100
    

How many generators had a status change in this quarter update? We shouldn't expect that many generators to have status changes.

Merge the quarters together using the generator ID. Each quarter should only have one record for each generator so the merge should be one to one.


In [3]:
previous_quarter = grab_previous_quarter(eia860m)
current_quarter = grab_current_quarter(eia860m)

In [4]:
previous_quarter.report_date.max(), current_quarter.report_date.max()

(Timestamp('2024-12-01 00:00:00'), Timestamp('2025-03-01 00:00:00'))

In [5]:
merged_quarters = previous_quarter.merge(current_quarter, on=["generator_id", "plant_id_eia"], validate="1:1", suffixes=("_previous", "_current"))

different_status_codes = merged_quarters["operational_status_code_previous"].ne(merged_quarters["operational_status_code_current"])
different_status_codes.value_counts()


False    36360
True       629
dtype: int64

For the generators have have a different status in the new update, check to see if the status change makes sense: ("Operational to Retired", "Under Construction to Operational", etc). A highlevel check to make sure the status changes make sense is to see if the status code numbers stay the same or increase. Higher number operational codes represent more advanced stages in a generator's life cycle.

In [6]:
new_status_code_is_greater = merged_quarters["operational_status_code_previous"].le(merged_quarters["operational_status_code_current"])

new_status_code_is_greater.value_counts()

True     36974
False       15
dtype: int64

In [7]:
merged_quarters[~new_status_code_is_greater][["raw_operational_status_code_previous", "raw_operational_status_code_current"]]

,raw_operational_status_code_previous,raw_operational_status_code_current
703,RE,OA
2085,RE,OS
2207,RE,OS
7483,RE,OS
16781,L,P
16782,L,P
16826,T,L
20619,RE,OA
21578,L,P
21711,L,P


Looks like there are a handful of generators that came out of retirement. Let dig into the status codes of the generators that have a new status in the udpated data.

In [8]:
pd.set_option('display.max_colwidth', None)

pudl_eia860m_status_codes

,code,status,description
0,OT,99,proposed
1,IP,98,"Planned new indefinitely postponed, or no longer in resource plan"
2,P,1,Planned for installation but regulatory approvals not initiated; Not under construction
3,L,2,Not under construction but site preparation could be underway
4,T,3,Regulatory approvals received. Not under construction but site preparation could be underway
5,U,4,"Under construction, less than or equal to 50 percent complete (based on construction time to date of operation)"
6,V,5,"Under construction, more than 50 percent complete (based on construction time to date of operation)"
7,TS,6,"Construction complete, but not yet in commercial operation (including low power testing of nuclear units). Operating under test conditions."
8,OA,7,Was not used for some or all of the reporting period but is expected to be returned to service in the next calendar year.
9,OP,7,"Operating (in commercial service or out of service within 365 days). For generators, this means in service (commercial operation) and producing some electricity. Includes peaking units that are run on an as needed (intermittent or seasonal) basis."


In [9]:
merged_quarters[different_status_codes][["raw_operational_status_code_previous", "raw_operational_status_code_current"]].value_counts()

raw_operational_status_code_previous  raw_operational_status_code_current
OP                                    RE                                     173
V                                     OP                                      82
U                                     V                                       82
TS                                    OP                                      43
SB                                    RE                                      39
OS                                    RE                                      31
V                                     TS                                      26
U                                     OP                                      24
P                                     U                                       17
                                      L                                       16
T                                     U                                       14
L                                  

Look at capacity change for each status code.

In [10]:
pct_change_mw_by_status(eia860m)

operational_status_code
1     10.151463
2      4.898029
3      2.522753
4     18.900128
5     12.782260
6     14.288642
7      0.831132
8      0.517567
99    36.065070
Name: capacity_mw, dtype: float64

## Capacity by status by ISO

In [11]:
ISO_REGIONS = ("MISO", "PJM", "CISO", "ERCO", "ISNE", "NYIS", "SWPP")

In [12]:
eia860_isos = eia860m[eia860m.balancing_authority_code_eia.isin(ISO_REGIONS)]


for region in ISO_REGIONS:
    print(region)
    pct_change = pct_change_mw_by_status(eia860_isos[eia860_isos["balancing_authority_code_eia"] == region])
    print(pct_change)
    print()

MISO
operational_status_code
1    -0.809216
2    -2.594637
3    -7.934785
4    22.470016
5    11.946750
6   -39.186941
7     1.272182
8     0.377111
Name: capacity_mw, dtype: float64

PJM
operational_status_code
1      30.772787
2       5.333808
3     -21.294235
4      40.156737
5      53.387270
6     483.636364
7       0.957162
8      -0.476059
99      0.000000
Name: capacity_mw, dtype: float64

CISO
operational_status_code
1      -4.713238
2       2.255213
3      63.063106
4      -8.989714
5      27.409887
6     110.473162
7       1.000050
8       0.755896
99      0.000000
Name: capacity_mw, dtype: float64

ERCO
operational_status_code
1      10.097776
2       4.259967
3       2.900674
4      34.841510
5      -4.253696
6      13.319210
7       1.747042
8       0.106738
99    941.176471
Name: capacity_mw, dtype: float64

ISNE
operational_status_code
1     -3.848114
2     -9.882462
3     91.876347
4      0.262902
5     -6.776448
6    -88.100209
7      0.761564
8      0.927302
99       